In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/us-patent-phrase-to-phrase-matching/sample_submission.csv
/kaggle/input/us-patent-phrase-to-phrase-matching/train.csv
/kaggle/input/us-patent-phrase-to-phrase-matching/test.csv


In [2]:
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')


In [3]:
!pip install kaggle

In [4]:
creds = '{"username":"himanshu210100","key":"a53a0538d26f6eda76c610d10bde3a73"}'

In [5]:
from pathlib import Path

cred_path = Path('~/.kaggle/kaggle.json').expanduser()
if not cred_path.exists():
    cred_path.parent.mkdir(exist_ok=True)
    cred_path.write_text(creds)
    cred_path.chmod(0o600)

In [6]:
path = Path('us-patent-phrase-to-phrase-matching')


In [7]:
if not iskaggle and not path.exists():
    import zipfile,kaggle
    kaggle.api.competition_download_cli(str(path))
    zipfile.ZipFile(f'{path}.zip').extractall(path)


In [8]:
if iskaggle:
    path = Path('../input/us-patent-phrase-to-phrase-matching')
    ! pip install -q datasets

In [9]:
!ls {path}

sample_submission.csv  test.csv  train.csv


In [10]:
df = pd.read_csv(path/'train.csv')

In [11]:
df


,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00
...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50
36471,756ec035e694722b,wood article,wooden material,B44,0.75


In [12]:
df.describe(include = 'object')

,id,anchor,target,context
count,36473,36473,36473,36473
unique,36473,733,29340,106
top,37d61fd2272659b1,component composite coating,composition,H01
freq,1,152,24,2186


In [13]:
df['input'] = 'TEXT1: ' + df.context + ';TEXT2: ' + df.target + ';ANC1 : ' + df.anchor 

In [14]:
df.input.head()

0    TEXT1: A47;TEXT2: abatement of pollution;ANC1 ...
1    TEXT1: A47;TEXT2: act of abating;ANC1 : abatement
2    TEXT1: A47;TEXT2: active catalyst;ANC1 : abate...
3    TEXT1: A47;TEXT2: eliminating process;ANC1 : a...
4     TEXT1: A47;TEXT2: forest region;ANC1 : abatement
Name: input, dtype: object

In [15]:
from datasets import Dataset,DatasetDict

ds = Dataset.from_pandas(df)

In [16]:
ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'score', 'input'],
    num_rows: 36473
})

In [17]:
model_nm = 'microsoft/deberta-v3-small'

In [18]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer
tokz = AutoTokenizer.from_pretrained(model_nm)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/opt/conda/lib/python3.7/site-packages/transformers/convert_slow_tokenizer.py:447: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [19]:
tokz.tokenize("How's the weather today? Maa don't worry bout' it ." )

['▁How',
 "'",
 's',
 '▁the',
 '▁weather',
 '▁today',
 '?',
 '▁Maa',
 '▁don',
 "'",
 't',
 '▁worry',
 '▁bout',
 "'",
 '▁it',
 '▁.']

In [20]:
tokz.tokenize("there is so much pollution")

['▁there', '▁is', '▁so', '▁much', '▁pollution']

In [21]:
def tok_func(x): return tokz(x["input"])

In [22]:
tok_ds = ds.map(tok_func, batched=True)

  0%|          | 0/37 [00:00<?, ?ba/s]

In [23]:
row = tok_ds[0]
row['input'], row['input_ids']

('TEXT1: A47;TEXT2: abatement of pollution;ANC1 : abatement',
 [1,
  54453,
  435,
  294,
  336,
  5753,
  346,
  104917,
  445,
  294,
  47284,
  265,
  6435,
  346,
  64097,
  435,
  877,
  47284,
  2])

In [24]:
tokz.vocab['▁pollution']

6435

In [25]:
tok_ds = tok_ds.rename_columns({'score':'labels'})

In [26]:
dds = tok_ds.train_test_split(0.25, seed=42)
dds

DatasetDict({
    train: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 27354
    })
    test: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9119
    })
})

In [27]:
def corr(x,y): return np.corrcoef(x,y)[0][1]


In [28]:
def show_corr(df, a, b):
    x,y = df[a],df[b]
    plt.scatter(x,y, alpha=0.5, s=4)
    plt.title(f'{a} vs {b}; r: {corr(x, y):.2f}')

In [29]:
def corr_d(eval_pred): return {'pearson': corr(*eval_pred)}


In [30]:
from transformers import TrainingArguments,Trainer

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [31]:
bs = 128
epochs = 4

In [32]:
lr = 8e-5


In [33]:
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')

In [34]:
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                  tokenizer=tokz, compute_metrics=corr_d)

Some weights of the model checkpoint at microsoft/deberta-v3-small were not used when initializing DebertaV2ForSequenceClassification: ['lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.classifier.weight', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.dense.weight', 'mask_predictions.LayerNorm.weight', 'mask_predictions.classifier.bias']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from 

In [35]:
trainer.train();


The following columns in the training set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: id, input, anchor, target, context. If id, input, anchor, target, context are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 27354
  Num Epochs = 4
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 1
  Total optimization steps = 428
  Number of trainable parameters = 141895681
You're using a DebertaV2TokenizerFast tokenizer. Please note that wi

Epoch,Training Loss,Validation Loss,Pearson
1,No log,0.031285,0.784152
2,No log,0.023266,0.819760
3,No log,0.022939,0.827607
4,No log,0.023369,0.829759


The following columns in the evaluation set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: id, input, anchor, target, context. If id, input, anchor, target, context are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9119
  Batch size = 512
The following columns in the evaluation set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: id, input, anchor, target, context. If id, input, anchor, target, context are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9119
  Batch size = 512
The following columns in the evaluation set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: id, input, anchor, target, context. If id, input,

In [36]:
Test = pd.read_csv(path/'test.csv')

In [37]:
Test

,id,anchor,target,context
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02
1,09e418c93a776564,adjust gas flow,altering gas flow,F23
2,36baf228038e314b,lower trunnion,lower locating,B60
3,1f37ead645e7f0c8,cap component,upper portion,D06
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04
5,474c874d0c07bd21,dry corn,dry corn starch,C12
6,442c114ed5c4e3c9,tunneling capacitor,capacitor housing,G11
7,b8ae62ea5e1d8bdb,angular contact bearing,contact therapy radiation,B23
8,faaddaf8fcba8a3f,produce liquid hydrocarbons,produce a treated stream,C10
9,ae0262c02566d2ce,diesel fuel tank,diesel fuel tanks,F02


In [38]:
Test['input'] = 'TEXT1: ' + Test.context + '; TEXT2: ' + Test.target + '; ANC1: ' + Test.anchor
eval_ds = Dataset.from_pandas(Test).map(tok_func, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [39]:
preds = trainer.predict(eval_ds).predictions.astype(float)
preds

The following columns in the test set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: id, input, anchor, target, context. If id, input, anchor, target, context are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 36
  Batch size = 512


array([[ 0.45117188],
       [ 0.75341797],
       [ 0.45849609],
       [ 0.31738281],
       [-0.00912476],
       [ 0.51464844],
       [ 0.51220703],
       [ 0.03326416],
       [ 0.30175781],
       [ 1.08789062],
       [ 0.23925781],
       [ 0.28833008],
       [ 0.75292969],
       [ 0.74462891],
       [ 0.75146484],
       [ 0.375     ],
       [ 0.27270508],
       [ 0.01004791],
       [ 0.63476562],
       [ 0.30834961],
       [ 0.34472656],
       [ 0.20056152],
       [ 0.25244141],
       [ 0.23510742],
       [ 0.59570312],
       [-0.04705811],
       [-0.04522705],
       [-0.02476501],
       [-0.01586914],
       [ 0.70458984],
       [ 0.29199219],
       [ 0.02183533],
       [ 0.73046875],
       [ 0.45605469],
       [ 0.4675293 ],
       [ 0.25024414]])

In [40]:
preds = np.clip(preds, 0, 1)
preds

array([[0.45117188],
       [0.75341797],
       [0.45849609],
       [0.31738281],
       [0.        ],
       [0.51464844],
       [0.51220703],
       [0.03326416],
       [0.30175781],
       [1.        ],
       [0.23925781],
       [0.28833008],
       [0.75292969],
       [0.74462891],
       [0.75146484],
       [0.375     ],
       [0.27270508],
       [0.01004791],
       [0.63476562],
       [0.30834961],
       [0.34472656],
       [0.20056152],
       [0.25244141],
       [0.23510742],
       [0.59570312],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.70458984],
       [0.29199219],
       [0.02183533],
       [0.73046875],
       [0.45605469],
       [0.4675293 ],
       [0.25024414]])

In [41]:
import datasets

submission = datasets.Dataset.from_dict({
    'id': eval_ds['id'],
    'score': preds
})

submission.to_csv('submission.csv', index=False)

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

1035